# Chatbot Modeling

## Import Libs

In [2]:
# General
import json
import string
import pickle
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Deep Learning
import tensorflow as tf

# Word Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # Only needed for first time usage of NLTK

[nltk_data] Downloading package punkt to C:\Users\Judan Syamsul
[nltk_data]     Hadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Files for Preprocessing and Modeling

### Intent

In [3]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [4]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [5]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


### Stopwords

In [6]:
# Define stemmer and stopword
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [7]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

### Slang

In [8]:
slang = pd.read_csv('lexicon/slang ke semi baku.csv')

slang_replace = {}
for i, row in enumerate(slang['slang']):
    slang_replace[row] = slang['formal'].iloc[i]

In [9]:
baku = pd.read_csv('lexicon/slang ke baku.csv')

std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

## Preprocessing

In [10]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in (slang_replace|std_word_replace): 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
        # Replace slang with standard word
        elif kata in slang_replace:
            for kata_slang in slang_replace[kata].split():
                new_text.append(std_word_replace.get(kata_slang, kata_slang))
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [11]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [12]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

## Create Corpus

In [13]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [14]:
corpus_size = len(words)

In [15]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [16]:
sequence_length = int(round(df['length'].max(),0))

In [17]:
corpus_size, sequence_length

(314, 8)

## Label Encoding

In [18]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = tf.keras.utils.to_categorical(y_train)

In [19]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [20]:
len(le.classes_)

22

## Modeling

### Text Vectorization

In [21]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [22]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([18, 12,  9,  0,  0,  0,  0,  0], dtype=int64)>

### Word Embedding

In [23]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [24]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[ 0.01727624, -0.02078724, -0.01191875, -0.04903857,  0.00421154,
        -0.0209415 ,  0.01499659,  0.00696637, -0.0154827 , -0.02698214,
        -0.02629602,  0.0093345 , -0.01035144,  0.02435717, -0.02351571,
        -0.04749465],
       [ 0.00317055,  0.04783611,  0.03803532,  0.04476628, -0.04922102,
        -0.04927615,  0.01647028,  0.00841957,  0.00876125, -0.02338641,
        -0.03744482, -0.03674288,  0.03587948, -0.04208728, -0.0009889 ,
        -0.02805727],
       [ 0.01363112,  0.01204417,  0.01691138,  0.00525782,  0.043053  ,
        -0.00035746, -0.04072682, -0.01129373, -0.00494387,  0.04524865,
         0.01970483,  0.01119577,  0.02011167, -0.00766503, -0.02092525,
         0.03506035],
       [ 0.00102137, -0.02530444, -0.01831088,  0.01596122, -0.03577871,
        -0.00665568,  0.00961898, -0.04072139,  0.02003891, -0.04276066,
        -0.01266681,  0.02618278,  0.03372863,  0.04164473, -0.04291748,
         

### Layering

In [25]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

### Training

In [26]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.018580,0.997895
96,0.018288,1.000000
97,0.017965,0.997895
98,0.017596,1.000000
99,0.016952,1.000000


### Evaluating Model

In [27]:
model.evaluate(df['clean_input'], y_train)

15/15 [==============================] - 1s 1ms/step - loss: 0.0166 - categorical_accuracy: 1.0000


[0.016621695831418037, 1.0]

In [28]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        12
           biaya       1.00      1.00      1.00        27
          daftar       1.00      1.00      1.00        22
         dokumen       1.00      1.00      1.00        23
    error upload       1.00      1.00      1.00         9
             ktp       1.00      1.00      1.00        16
     link sosmed       1.00      1.00      1.00        17
        lowongan       1.00      1.00      1.00        17
   lupa password       1.00      1.00      1.00        21
            nama       1.00      1.00      1.00         9
        pas foto       1.00      1.00      1.00        16
       pengantar       1.00      1.00      1.00        10
         penutup       1.00      1.00      1.00        48
   qualification       1.00      1.00      1.00        20
responsibilities       1.00      1.00      1.00        21
          salary       1.00      1.00      1.00        37
            s

### Function Covering End-to-End Process

In [29]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()] # Get the index with highest probability
    i = 0
    try:
        if textvect(text).numpy().max() > 1: # If the input is known word(s)
            while i < len(intent_json['intents']):
                if res == intent_json['intents'][i]['intent']:
                    response = intent_json['intents'][i]['response']
                    break
                else:
                    i+=1
        else: # If only unknown word(s)
            response = ['Maaf, Kak. Aku tidak mengerti...']
    except: # If empty string or any error occured
        response = ['Maaf, Kak. Aku tidak mengerti...']

    # For debugging only
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])

    return print(np.random.choice(response))

In [30]:
tes1 = 'aku mau daftar rekrutmen'
clean_text(tes1)

'aku mau daftar rekrutmen'

In [31]:
model.predict([clean_text(tes1)])

array([[3.41042614e-05, 1.30600983e-03, 9.86958206e-01, 1.03317014e-07,
        4.25029830e-05, 1.78054597e-06, 1.15282319e-05, 4.96281078e-04,
        1.79441338e-06, 1.33625474e-06, 3.87672801e-04, 1.83891498e-05,
        4.06584368e-05, 1.02942213e-02, 1.78547980e-05, 4.76114110e-06,
        8.58166313e-05, 8.49789217e-07, 1.63138771e-04, 3.69350855e-05,
        1.54515453e-06, 9.44350322e-05]], dtype=float32)

In [32]:
model.predict([clean_text(tes1)]).argmax()

2

In [33]:
le.classes_[model.predict([clean_text(tes1)]).argmax()]

'daftar'

In [34]:
le.classes_[model.predict([clean_text(tes1)]).argmax()] == intent_json['intents'][2]['intent']

True

In [35]:
intent_json['intents'][2]['response']

['Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:\n1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara\n2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi\n3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di https://rekrutmen.fiktif.id/news atau email masing-masing']

In [36]:
bot_response('aku mau daftar rekrutmen')

[{'CV': 3.410426e-05}, {'biaya': 0.0013060098}, {'daftar': 0.9869582}, {'dokumen': 1.03317014e-07}, {'error upload': 4.2502983e-05}, {'ktp': 1.780546e-06}, {'link sosmed': 1.1528232e-05}, {'lowongan': 0.0004962811}, {'lupa password': 1.7944134e-06}, {'nama': 1.3362547e-06}, {'pas foto': 0.0003876728}, {'pengantar': 1.838915e-05}, {'penutup': 4.0658437e-05}, {'qualification': 0.010294221}, {'responsibilities': 1.7854798e-05}, {'salary': 4.761141e-06}, {'sapa': 8.581663e-05}, {'sertifikat': 8.497892e-07}, {'skck': 0.00016313877}, {'timeline': 3.6935086e-05}, {'training': 1.5451545e-06}, {'transkrip-ijazah': 9.443503e-05}]
daftar
Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:
1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara
2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi
3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutny

### Saving Model

In [37]:
# import pickle
# pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [38]:
# pickle.dump({'config': textvect.get_config(),
#              'weights': textvect.get_weights()}
#             , open("saved_model/textvect.pkl", "wb"))

In [39]:
# model.save('saved_model/model', save_traces=True)

## Inference

In [40]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([233,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [41]:
clean_text('ok')

'ok'

In [42]:
textvect('ok').numpy().max() > 1

True

In [43]:
bot_response('a')

[{'CV': 5.36883e-05}, {'biaya': 0.007395609}, {'daftar': 0.0016359695}, {'dokumen': 0.0008477761}, {'error upload': 1.575233e-07}, {'ktp': 0.0005843639}, {'link sosmed': 0.820636}, {'lowongan': 0.000102379825}, {'lupa password': 0.006315019}, {'nama': 0.00040211546}, {'pas foto': 0.0005634637}, {'pengantar': 1.5575746e-05}, {'penutup': 0.032228664}, {'qualification': 9.674149e-05}, {'responsibilities': 1.721717e-05}, {'salary': 0.029005488}, {'sapa': 0.043097135}, {'sertifikat': 0.0012886999}, {'skck': 0.00012526671}, {'timeline': 0.0003172845}, {'training': 0.019812303}, {'transkrip-ijazah': 0.035459265}]
link sosmed
Maaf, Kak. Aku tidak mengerti...


In [44]:
bot_response('ya')

[{'CV': 2.2069027e-07}, {'biaya': 0.0020134135}, {'daftar': 0.0003073075}, {'dokumen': 0.00043082391}, {'error upload': 7.895572e-10}, {'ktp': 2.39478e-07}, {'link sosmed': 0.00016925728}, {'lowongan': 4.198607e-05}, {'lupa password': 0.0004760598}, {'nama': 1.8830763e-06}, {'pas foto': 1.580732e-07}, {'pengantar': 5.6112373e-08}, {'penutup': 0.9905691}, {'qualification': 3.6861835e-05}, {'responsibilities': 1.4891566e-05}, {'salary': 0.0012615839}, {'sapa': 0.0016094416}, {'sertifikat': 3.580304e-08}, {'skck': 2.782781e-06}, {'timeline': 0.0018119157}, {'training': 0.0009757699}, {'transkrip-ijazah': 0.0002762013}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [45]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [46]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [47]:
bot_response('oh ok')

[{'CV': 1.6411526e-07}, {'biaya': 0.001864196}, {'daftar': 0.0002562904}, {'dokumen': 0.00016881076}, {'error upload': 1.2396235e-09}, {'ktp': 6.731565e-08}, {'link sosmed': 0.0001283172}, {'lowongan': 4.3904907e-05}, {'lupa password': 0.00038904496}, {'nama': 2.2123256e-06}, {'pas foto': 1.958348e-07}, {'pengantar': 1.151137e-07}, {'penutup': 0.9940013}, {'qualification': 5.80017e-05}, {'responsibilities': 1.7246173e-05}, {'salary': 0.0005495991}, {'sapa': 0.0009511915}, {'sertifikat': 1.4964932e-08}, {'skck': 1.2198878e-06}, {'timeline': 0.00093475165}, {'training': 0.0005533632}, {'transkrip-ijazah': 7.996002e-05}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [48]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 9.126592e-05}, {'biaya': 0.0018630566}, {'daftar': 0.00425789}, {'dokumen': 0.00014764487}, {'error upload': 1.8197965e-05}, {'ktp': 1.969937e-07}, {'link sosmed': 0.00029554655}, {'lowongan': 0.98559266}, {'lupa password': 0.00075157726}, {'nama': 4.9915594e-05}, {'pas foto': 8.569103e-07}, {'pengantar': 2.7303118e-05}, {'penutup': 0.0005332018}, {'qualification': 0.0030277157}, {'responsibilities': 0.00057007215}, {'salary': 1.834881e-05}, {'sapa': 0.000459054}, {'sertifikat': 5.6223794e-06}, {'skck': 6.5278204e-05}, {'timeline': 0.0022178087}, {'training': 2.9053028e-06}, {'transkrip-ijazah': 3.8835747e-06}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [49]:
bot_response('ingin')

[{'CV': 2.908049e-05}, {'biaya': 0.00029828688}, {'daftar': 0.07239391}, {'dokumen': 0.00013096702}, {'error upload': 3.0858308e-07}, {'ktp': 3.3702447e-05}, {'link sosmed': 0.0013146426}, {'lowongan': 0.0008550244}, {'lupa password': 0.0006279098}, {'nama': 6.914554e-05}, {'pas foto': 3.135934e-06}, {'pengantar': 3.3143408e-07}, {'penutup': 0.0639518}, {'qualification': 0.007311456}, {'responsibilities': 0.0013466298}, {'salary': 0.00044325012}, {'sapa': 0.832983}, {'sertifikat': 1.4019523e-06}, {'skck': 0.00026106357}, {'timeline': 0.0030226817}, {'training': 0.012111078}, {'transkrip-ijazah': 0.002811091}]
sapa
Maaf, Kak. Aku tidak mengerti...


In [50]:
bot_response('pas foto hilang gimana ya')

[{'CV': 0.000518769}, {'biaya': 0.00020780317}, {'daftar': 0.0006725984}, {'dokumen': 1.7531819e-06}, {'error upload': 0.00028842772}, {'ktp': 0.0059102927}, {'link sosmed': 0.000121205194}, {'lowongan': 4.2596633e-07}, {'lupa password': 1.3231415e-06}, {'nama': 1.707878e-05}, {'pas foto': 0.9885134}, {'pengantar': 0.002006357}, {'penutup': 3.562962e-06}, {'qualification': 4.9234124e-05}, {'responsibilities': 8.7609914e-07}, {'salary': 5.5123513e-05}, {'sapa': 8.473102e-06}, {'sertifikat': 0.0005949707}, {'skck': 0.000110772664}, {'timeline': 6.3338854e-07}, {'training': 0.0002761693}, {'transkrip-ijazah': 0.00064090214}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format f

ktp, pas foto, password masih ketuker2

In [51]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [52]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([153,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [53]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [54]:
data = {'user_input':'hai hai'}
data = json.dumps(data)
loaded_data = json.loads(data)

In [55]:
type(loaded_data)

dict

In [56]:
tespred = str(model.predict([clean_text(loaded_data['user_input'])]).argmax())

In [57]:
tes_res = {'pred':tespred}
tes_res = json.dumps(tes_res)
loaded_res = json.loads(tes_res)

In [58]:
le.classes_[int(loaded_res['pred'])]

'sapa'

In [59]:
loaded_data['user_input']

'hai hai'

In [60]:
1000

1000

In [61]:
str(1000)

'1000'

In [62]:
int(str(1000))

1000

In [63]:
le.classes_[int(str(10))]

'pas foto'

In [64]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([153,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [65]:
user_input = 'apa kualifikasi engineer'
clean_input = clean_text(user_input)
pred = model.predict([clean_input]).argmax()
if textvect(clean_input).numpy().max() <= 1:
    label_idx = 1000
else:
    label_idx = pred 
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']

print(np.random.choice(response))

Curriculum Vitae (CV) wajib diupload bersama dokumen lain ya, Kak. Isi CV bisa mengikuti contoh-contoh yang ada di internet dan tidak lebih dari 1 halaman serta berukuran tidak lebih dari 1 MB.


In [66]:
le.classes_[label_idx]

'CV'